# Crawler de dados do Google Shopping

Para a criação do _streamlit_ de precificação de coisas, vamos precisar criar um crawler da aba "Shopping" do Google. Para isso, primeiro temos que entender o padrão da URL para podermos fazer requisições.

Vamos ver uma URL:

``https://www.google.com/search?q=mesa+tampo+de+vidro&tbm=shop``

Vemos que temos que adicionar à raiz a pesquisa, substituindo espaço por +, e terminar com &tbm=shop.

In [46]:
import requests
import re

root = 'https://www.google.com.br/search?q='
query = 'mesa de madeira'
suffix = '&tbm=shop&hl=pt'
headers = {'User-agent': 'Mozilla/5.0'}

link = root + re.sub('\s', '+', query) + suffix

print(link)
results = requests.get(link, headers=headers).content
print(results[:100])

https://www.google.com.br/search?q=mesa+de+madeira&tbm=shop&hl=de
b'<!doctype html><html lang="de-BR"><head><title>mesa de madeira - Google Shopping</title><meta conten'


# Tratamento de dados

Agora, vamos extrair os dados da página e transformá-la num dataframe, com os produtos, as categorias e os preços.

In [48]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(results, 'html.parser')

divs_com_preco = [
    i for i in 
    soup.findAll('div')
    if len(i.findAll('div')) >= 6
    if len([
        j for j in i.findAll('div')
        if 'R$' in str(j)
    ]) > 0
    and len([
        j for j in i.findAll('div')
        if 'de R$' in str(j)
    ]) == 0
]

print(len(divs_com_preco))
print(divs_com_preco[-1].prettify()[-100:])

27
Livre
   </div>
   <span class="dD8iuc">
    Kostenloser Versand
   </span>
  </div>
 </div>
</div>



In [3]:
import pandas as pd
cols = [
        'site', 'titulo', 'preco', 'rating', 'imagem'
]
produtos = pd.DataFrame(
    columns = cols
)
produtos.head()

,site,titulo,preco,rating,imagem


In [4]:
for div in divs_com_preco:
    vetor = []
    for a in div.findAll('a'):
        if len(a.findAll()) == 0:
            # site
            vetor.append(re.findall('https?://[^\/]+', str(a['href']))[0])
        if 'aria-hidden' not in a.attrs.keys() and len(a.findAll('span')) == 0:
            # titulo
            vetor.append(a.text)
    for span in div.findAll('span'):
        if 'R$' in span.text:
            # preco
            vetor.append(
                    float(
                        re.sub(',', '.', 
                            re.sub('\.', '', 
                                ''.join(
                                    re.findall('[0123456789\.,]', span.text)
                                )
                            )
                        )
                    )
                )
    # rating
    rating = div.text.count('★')
    if rating == 0:
        rating = None
    vetor.append(rating)
    for img in div.findAll('img'):
        # imagem
        vetor.append(str(img['src']))
    
    vetor_df = pd.DataFrame(vetor).T
    vetor_df.columns = cols
    produtos = produtos.append(vetor_df, ignore_index=True)

In [5]:
produtos

,site,titulo,preco,rating,imagem
0,https://www.submarino.com.br,Mesa de madeira (Estrutura 100% madeira),149.99,None,https://encrypted-tbn0.gstatic.com/shopping?q=...
1,https://www.magazineluiza.com.br,Mesa De Jantar Livia 90cm Base Madeira Maciça ...,279.93,None,https://encrypted-tbn0.gstatic.com/shopping?q=...
2,https://www.madeirado.com.br,Mesa de madeira de demolição com Bordas Orgâni...,3990.0,None,https://encrypted-tbn0.gstatic.com/shopping?q=...
3,https://www.panoramamoveis.com.br,Mesa De Madeira Imperial Canela Rústico - Art ...,438.81,5,https://encrypted-tbn3.gstatic.com/shopping?q=...
4,https://www.panoramamoveis.com.br,Mesa De Madeira Imperial Canela Rústico - Art ...,438.81,5,https://encrypted-tbn3.gstatic.com/shopping?q=...
5,https://produto.mercadolivre.com.br,Mesa De Jantar Retangular 4 Lugares Madeira Mo...,334.99,5,https://encrypted-tbn2.gstatic.com/shopping?q=...
6,https://produto.mercadolivre.com.br,Mesa De Jantar Retangular 4 Lugares Madeira Mo...,334.99,5,https://encrypted-tbn2.gstatic.com/shopping?q=...
7,https://www.madeiramadeira.com.br,Mesa De Jantar Berlin Com Base De Madeira Maci...,327.25,None,https://encrypted-tbn2.gstatic.com/shopping?q=...
8,https://www.casasbahia.com.br,Mesa De Jantar 6 Lugares Madeira Maciça Mandac...,699.9,5,https://encrypted-tbn1.gstatic.com/shopping?q=...
9,https://www.casasbahia.com.br,Mesa De Jantar 6 Lugares Madeira Maciça Mandac...,699.9,5,https://encrypted-tbn1.gstatic.com/shopping?q=...


# Pipeline de pesquisa e tratamento de dados

A última coisa que precisaremos fazer é criar algumas funções para que o pipeline de busca e tratamento de dados seja automático.

In [9]:
!pip freeze

absl-py==1.0.0
altair==4.1.0
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
astor==0.8.1
astroid==2.4.2
astunparse==1.6.3
atomicwrites==1.4.0
attrs==20.3.0
backcall==0.2.0
base58==2.1.1
beautifulsoup4==4.9.3
bleach==4.1.0
blinker==1.4
blis==0.7.4
boto3==1.17.9
botocore==1.20.9
bs4==0.0.1
cachetools==5.0.0
catalogue==2.0.1
certifi==2021.10.8
cffi==1.15.0
chardet==4.0.0
charset-normalizer==2.0.12
chess==1.9.3
click==7.1.2
cloudpickle==2.0.0
colorama==0.4.4
commonmark==0.9.1
cutlet==0.1.19
cycler==0.10.0
cymem==2.0.5
Cython==0.29.25
debugpy==1.5.1
decorator==5.1.0
defusedxml==0.7.1
distlib==0.3.6
en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0-py3-none-any.whl
entrypoints==0.3
et-xmlfile==1.1.0
filelock==3.8.0
Flask==2.0.2
flatbuffers==2.0
freetype-py==2.2.0
fugashi==1.1.2
gast==0.5.3
gitdb==4.0.9
GitPython==3.1.24
glfw==2.1.0
google-auth==2.6.5
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
GPUtil==1.4.0
grpcio==1.

In [6]:
def search(product):
    root = 'https://www.google.com/search?q='
    suffix = '&tbm=shop'

    link = root + re.sub('\s', '+', product) + suffix
    return requests.get(link).content

def get_products(html):
    soup = BeautifulSoup(html, 'html.parser')

    divs_com_preco = [
        i for i in 
        soup.findAll('div')
        if len(i.findAll('div')) >= 6
        if len([
            j for j in i.findAll('div')
            if 'R$' in str(j)
        ]) > 0
        and len([
            j for j in i.findAll('div')
            if 'de R$' in str(j)
        ]) == 0
    ]
    cols = [
        'site', 'titulo', 'preco', 'rating', 'imagem'
    ]
    produtos = pd.DataFrame(
        columns = cols
    )

    for div in divs_com_preco:
        vetor = []
        for a in div.findAll('a'):
            if len(a.findAll()) == 0:
                # site
                vetor.append(re.findall('https?://[^\/]+', str(a['href']))[0])
            if 'aria-hidden' not in a.attrs.keys() and len(a.findAll('span')) == 0:
                # titulo
                vetor.append(a.text)
        for span in div.findAll('span'):
            if 'R$' in span.text:
                # preco
                vetor.append(
                    float(
                        re.sub(',', '.', 
                            re.sub('\.', '', 
                                ''.join(
                                    re.findall('[0123456789\.,]', span.text)
                                )
                            )
                        )
                    )
                )
        # rating
        rating = div.text.count('★')
        if rating == 0:
            rating = None
        vetor.append(rating)
        for img in div.findAll('img'):
            # imagem
            vetor.append(str(img['src']))
        
        vetor_df = pd.DataFrame(vetor).T
        vetor_df.columns = cols
        produtos = produtos.append(vetor_df, ignore_index=True)
    return produtos

In [7]:
df = get_products(
    search('capivara de pelúcia')
)
df

,site,titulo,preco,rating,imagem
0,https://shopee.com.br,Brinquedo Pelúcia Capivara,20.0,None,https://encrypted-tbn1.gstatic.com/shopping?q=...
1,https://s.click.aliexpress.com,mulação de brinquedos de pelúcia de capivara p...,0.53,None,https://encrypted-tbn0.gstatic.com/shopping?q=...
2,https://s.click.aliexpress.com,Bonito Simulação Capivara Brinquedos de Pelúci...,5.1,None,https://encrypted-tbn1.gstatic.com/shopping?q=...
3,https://shopee.com.br,NAKAZAWA Capivara Macio Decoração de Casa Anim...,40.0,None,https://encrypted-tbn3.gstatic.com/shopping?q=...
4,https://shopee.com.br,Simulação Capivara Brinquedo de pelúcia Bonito...,41.47,None,https://encrypted-tbn2.gstatic.com/shopping?q=...
5,https://www.lojario.com.br,Capivara de Pelúcia 1 Unidade,119.9,None,https://encrypted-tbn0.gstatic.com/shopping?q=...
6,https://produto.mercadolivre.com.br,Capivara Fofinha Brinquedos De Pelúcia Meninos...,59.58,None,https://encrypted-tbn1.gstatic.com/shopping?q=...
7,https://www.americanas.com.br,Brinquedos De Pelucia Capivara Adorável Fofa G...,69.99,None,https://encrypted-tbn3.gstatic.com/shopping?q=...
8,https://solucaotechstore.com.br,Capivara de pelúcia 18cm,80.0,None,https://encrypted-tbn0.gstatic.com/shopping?q=...
9,https://www.magazineluiza.com.br,Brinquedos De Pelucia Capivara Adorável Fofa -...,122.0,None,https://encrypted-tbn1.gstatic.com/shopping?q=...


In [8]:
import plotly.express as px
import numpy as np
px.histogram(df.preco)

In [62]:
valores_hist = np.histogram(df.preco)
filtro = valores_hist[1] <= df.preco.median()
valores_hist[1][filtro]

array([22.0], dtype=object)

In [64]:
valores_hist[1]

array([22.0, 79.7, 137.4, 195.10000000000002, 252.8, 310.5,
       368.20000000000005, 425.90000000000003, 483.6, 541.3000000000001,
       599.0], dtype=object)